In [1]:
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

map_location = torch.device('cuda:0')

In [2]:
import os
import shutil
import nip

path_to_config = '/home/msst/repo/Quantization/configs/vq/llama2_7b/trellis/llama2_trellis_128seq_reassigns.yaml'

config = nip.load(path_to_config)
logdir = config['logdir']
os.makedirs(logdir, exist_ok=True)
try:
    shutil.copy(path_to_config, logdir)
except shutil.SameFileError:
    pass


fp_blocks_path = config['fp_blocks_path']
q_blocks_path = config['q_blocks_path']
trained_q_blocks_path = os.path.join(config['logdir'], 'per_block_q_trained')

os.makedirs(trained_q_blocks_path, exist_ok=True)

In [3]:
trainer = qlib.HomequantTrainerPTQ(
    logdir = config['logdir'],
    training_config = config['training_params'],
    tokenizer=qlib.load_tokenizer(config['model_name']),
    rotary_emb=torch.load(f'{config["fp_blocks_path"]}/model.rotary_emb',),
	quant_classes=[qlib.TrellisLinear],
	store_dtype=torch.float16
)

/tmp/ipykernel_2269061/3086267953.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rotary_emb=torch.load(f'{config["fp_blocks_path"]}/model.rotary_emb',),


Resolving data files:   0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
print("model_decoder.embed_tokens")
trainer.finetune_block_ptq(
        path_to_fp_block = f'{fp_blocks_path}/model.embed_tokens',
        path_to_q_block = f'{q_blocks_path}/model.embed_tokens',
        path_to_q_block_trained = f'{trained_q_blocks_path}/model.embed_tokens',
        train=False,
        with_input_preparation=False,
    )

for layer_idx in range(32):
    print()
    print('layer_idx:', layer_idx)
        
    trainer.logger.block_label = f'block.{layer_idx}'
    trainer.finetune_block_ptq(
        path_to_fp_block = f'{fp_blocks_path}/model.layers.{layer_idx}',
        path_to_q_block = f'{q_blocks_path}/model.layers.{layer_idx}',
        path_to_q_block_trained = f'{trained_q_blocks_path}/model.layers.{layer_idx}',
        train=True,
        with_input_preparation=True
    )

model_decoder.embed_tokens


/home/msst/repo/Quantization/qlib/ptq/homequant_ptq_train.py:222: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  block = torch.load(path_to_fp_block, map_location=self.device


layer_idx: 0


collecting activations: 100%|██████████| 16/16 [00:03<00:00,  5.16it/s]
/home/msst/repo/Quantization/qlib/ptq/homequant_ptq_utils.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental f


layer_idx: 1


collecting activations: 100%|██████████| 16/16 [00:03<00:00,  5.24it/s]
/home/msst/repo/Quantization/qlib/ptq/homequant_ptq_train.py:227: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

In [2]:
import torch

b1 = torch.load('/home/msst/repo/Quantization/logs/llama2-7b/trellis/llama2_trellis_128seq_reassigns/per_block_q_trained/model.layers.0')
b2 = torch.load('/home/msst/repo/Quantization/logs/llama2-7b/trellis/llama2_trellis_128seq/per_block_q_trained/model.layers.0')

/tmp/ipykernel_2277463/383761143.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  b1 = torch.load('/home/msst/repo/Quantization/logs/llama2-7b/trellis/llama2_trellis_128s

In [3]:
submodule_names = [
	"self_attn.q_proj",
	"self_attn.k_proj",
	"self_attn.v_proj",
	"self_attn.o_proj",
	'mlp.gate_proj',
	'mlp.up_proj',
	'mlp.down_proj',
]
for submodule_name in submodule_names:
    trellis1 = b1.get_submodule(submodule_name).trellis
    trellis2 = b2.get_submodule(submodule_name).trellis

    print(submodule_name, (trellis1 != trellis2).sum() // 256)

self_attn.q_proj tensor(1575, device='cuda:0')
self_attn.k_proj tensor(1714, device='cuda:0')
self_attn.v_proj tensor(2273, device='cuda:0')
self_attn.o_proj tensor(1825, device='cuda:0')
mlp.gate_proj tensor(4668, device='cuda:0')
mlp.up_proj tensor(4755, device='cuda:0')
mlp.down_proj tensor(4257, device='cuda:0')
